# Classification de texte avec Word2Vec et Régression Logistique

Ce notebook explique en détail un pipeline complet de classification de texte utilisant le prétraitement linguistique avec SpaCy, l'embedding Word2Vec, et un modèle de régression logistique. 

Nous allons explorer chaque étape, expliquer les concepts mathématiques et algorithmiques, et montrer comment les données textuelles sont transformées en vecteurs numériques pour permettre l'apprentissage automatique.

---

## Concepts clés

- **Traitement automatique du langage naturel (TALN)** : Ensemble de techniques permettant à une machine de comprendre et manipuler du texte humain.
- **Word2Vec** : Méthode d'embedding qui transforme chaque mot en un vecteur dense, capturant des relations sémantiques et syntaxiques.
- **Régression Logistique** : Modèle de classification supervisée qui prédit la probabilité d'appartenance à une classe.
- **Lemmatisation** : Réduction des mots à leur forme de base (ex : "mangeaient" → "manger").
- **Stopwords** : Mots fréquents sans valeur informative (ex : "le", "et", "de").

---

In [ ]:
# Importation des bibliothèques nécessaires
import pandas as pd
import re
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from gensim.models import Word2Vec
import json
import numpy as np

## Chargement et exploration des données

Nous chargeons les données d'entraînement au format JSON. Chaque ligne correspond à un texte à classer. Nous vérifions la présence de valeurs manquantes dans la colonne 'description'.

In [ ]:
version_ = "V.1.0.8"
print("Version:", version_)
print("Loading training data...")
with open('train.json', 'r', encoding='utf-8') as f:
    train_data = json.load(f)
df = pd.DataFrame(train_data)
print("Checking for problematic rows in 'description'...")
print(df['description'].isnull().sum(), "rows are null in 'description'")
print(df['description'].head())
df['description'] = df['description'].fillna("")

## Prétraitement linguistique avec SpaCy

Nous utilisons SpaCy pour nettoyer et lemmatiser les textes. 
Le nettoyage consiste à :
- Supprimer les balises HTML
- Supprimer les URLs
- Mettre en minuscules
- Lemmatiser chaque mot
- Supprimer les stopwords et les tokens non alphabétiques

La lemmatisation permet de réduire la dimensionnalité et d'améliorer la généralisation du modèle.

In [ ]:
print("Loading Spacy model...")
nlp = spacy.load('en_core_web_sm')

print("Cleaning and tokenizing training data...")
df['description'] = df['description'].fillna("")

def clean_and_tokenize(text):
    if not isinstance(text, str) or not text.strip():
        return "", []  # Retourne des valeurs vides si le texte est invalide
    text = re.sub(r'<[^>]+>', ' ', text)  # Suppression des balises HTML
    text = re.sub(r'http\S+', ' ', text)  # Suppression des URLs
    text = text.lower().strip()  # Mise en minuscules
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]  # Lemmatisation et suppression des stopwords
    return " ".join(tokens), tokens

Nous appliquons la fonction de nettoyage à chaque description. 
Chaque texte est transformé en deux colonnes :
- `Clean` : texte nettoyé et lemmatisé (pour d'autres usages)
- `Tokens` : liste des tokens (mots) pour l'entraînement Word2Vec

In [ ]:
print("Cleaning and tokenizing training data...")
results = df['description'].apply(clean_and_tokenize)
tuples = results.apply(lambda x: x if (isinstance(x, tuple) and len(x)==2) else ("",[]))
results_df = pd.DataFrame(tuples.tolist(), index=results.index, columns=['Clean', 'Tokens'])
df[['Clean', 'Tokens']] = results_df
print(df.columns)
print(df.head())

## Embedding des textes avec Word2Vec

**Word2Vec** est un algorithme non supervisé qui apprend à représenter chaque mot par un vecteur dense de dimension fixe (ici 100). 
L'idée est que des mots ayant des contextes similaires auront des vecteurs proches dans l'espace vectoriel.

Il existe deux architectures principales :
- **CBOW (Continuous Bag of Words)** : prédit un mot à partir de son contexte
- **Skip-gram** : prédit le contexte à partir d'un mot (ici `sg=1` pour skip-gram)

Mathématiquement, Word2Vec cherche à maximiser la probabilité de prédire les mots de contexte autour d'un mot cible, ce qui revient à factoriser la matrice de co-occurrence des mots.

Après entraînement, chaque mot est représenté par un vecteur de taille 100. Pour représenter un texte, on fait la moyenne des vecteurs des mots présents dans le texte.

In [ ]:
print("Training Word2Vec model...")
model = Word2Vec(
    size=100,  # Taille des vecteurs de mots
    window=5,  # Fenêtre de contexte
    min_count=2,  # Ignore les mots trop rares
    workers=4,    # Nombre de threads
    sg=1,         # Skip-gram
    iter=10       # Nombre d'itérations
)
model.build_vocab(df['Tokens'])
model.train(df['Tokens'], total_examples=model.corpus_count, epochs=model.iter)

## Chargement des labels (catégories)

Nous associons à chaque texte sa catégorie cible pour l'apprentissage supervisé.

In [ ]:
print("Loading labels...")
labels_df = pd.read_csv('train_label.csv')
df = df.merge(labels_df, on='Id')

## Vectorisation des textes par moyenne des vecteurs Word2Vec

Pour chaque texte, on calcule la moyenne des vecteurs Word2Vec de ses mots. 
Cela donne un vecteur de taille 100 représentant le texte entier.

Si un mot n'est pas dans le vocabulaire Word2Vec, il est ignoré. Si aucun mot n'est connu, on retourne un vecteur nul.

In [ ]:
print("Vectorizing tokens using Word2Vec...")
def vectorize_tokens(tokens, model):
    vectors = [model.wv[token] for token in tokens if token in model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

df['W2V_Vector'] = df['Tokens'].apply(lambda tokens: vectorize_tokens(tokens, model))

## Préparation des données pour l'apprentissage supervisé

On construit la matrice X (features) et le vecteur y (labels) pour entraîner le modèle de classification.

In [ ]:
print("Preparing training and testing data...")
X = np.vstack(df['W2V_Vector'].values)
y = df['Category'].values

## Entraînement du modèle de Régression Logistique

La **régression logistique** est un modèle linéaire de classification. 
Elle calcule la probabilité d'appartenance à chaque classe à partir d'une combinaison linéaire des features (ici, les vecteurs Word2Vec).

Mathématiquement, pour chaque classe $k$ :
$$ P(y=k|x) = \frac{\exp(w_k^T x + b_k)}{\sum_j \exp(w_j^T x + b_j)} $$
où $x$ est le vecteur du texte, $w_k$ et $b_k$ sont les paramètres appris pour la classe $k$.

Le modèle est entraîné pour maximiser la vraisemblance des labels sur l'ensemble d'entraînement.

In [ ]:
print("Training Logistic Regression model...")
clf_w2v = LogisticRegression(max_iter=1000)
clf_w2v.fit(X, y)

## Préparation et vectorisation des données de test

On applique le même prétraitement et la même vectorisation aux textes de test.

In [ ]:
print("Loading test data...")
with open('test.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)
test_df = pd.DataFrame(test_data)

print("Cleaning and tokenizing test data...")
test_df[['Clean', 'Tokens']] = test_df['description'].apply(lambda t: pd.Series(clean_and_tokenize(t)))

print("Vectorizing test data using Word2Vec...")
test_df['W2V_Vector'] = test_df['Tokens'].apply(lambda tokens: vectorize_tokens(tokens, model))
X_test_w2v = np.vstack(test_df['W2V_Vector'].values)

## Prédiction des catégories sur les données de test

Le modèle prédit la catégorie la plus probable pour chaque texte de test.

In [ ]:
print("Predicting categories for test data...")
test_df['Predicted_Category_W2V'] = clf_w2v.predict(X_test_w2v)

## Génération du fichier de soumission

On prépare le fichier de soumission au format attendu, associant chaque Id de test à la catégorie prédite.

In [ ]:
print("Preparing submission file...")
template = pd.read_csv('template_submissions.csv')
template['Category'] = template['Id'].map(
    test_df.set_index('Id')['Predicted_Category_W2V']
)
template.to_csv('submission.csv', index=False)
print("Submission file saved as 'submission.csv'.")
print("Version:", version_)

---

## Conclusion et concepts mathématiques

- **Word2Vec** permet de transformer le texte en vecteurs denses, où la proximité géométrique reflète la similarité sémantique.
- La **moyenne des vecteurs** pour chaque texte permet d'obtenir une représentation globale, adaptée à la classification.
- La **régression logistique** sépare les classes dans l'espace vectoriel à l'aide d'hyperplans, en maximisant la probabilité des labels observés.

Ce pipeline est un exemple classique de l'application de l'intelligence artificielle au traitement du langage naturel :
1. Prétraitement linguistique
2. Transformation des textes en vecteurs numériques
3. Apprentissage supervisé pour la classification

Ce type d'approche est utilisé dans la détection de spam, la modération de contenu, la recommandation de documents, etc.